<a href="https://colab.research.google.com/github/simecek/dspracticum2023/blob/main/lesson05/FineTuningTransformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#pip install -qq transformers[torch] datasets accelerate

In [4]:
from datasets import load_dataset


dataset = load_dataset(
    "yelp_review_full",
)

dataset

Generating train split:   0%|          | 0/650000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})

In [5]:
from transformers import AutoTokenizer

MODEL = "distilbert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=True, return_tensors="pt")

train_dataset = dataset['train'].select(range(500)).map(preprocess_function, batched=True)
valid_dataset = dataset['train'].select(range(500,1000)).map(preprocess_function, batched=True)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [6]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL, num_labels=5  # if you change the dataset, do not forget to update
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'classifier.weight', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
from transformers import TrainingArguments, Trainer
from sklearn.metrics import accuracy_score
import numpy as np

def compute_metrics(p):
    logits = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.argmax(logits, axis=1)
    return {"accuracy": accuracy_score(p.label_ids, preds)}

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=4,
    per_device_train_batch_size=16,
    evaluation_strategy="epoch",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    compute_metrics=compute_metrics
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.325785,0.428000
2,No log,1.181671,0.510000
3,No log,1.176395,0.488000
4,No log,1.133225,0.536000


TrainOutput(global_step=128, training_loss=1.035851001739502, metrics={'train_runtime': 122.4184, 'train_samples_per_second': 16.337, 'train_steps_per_second': 1.046, 'total_flos': 264948971520000.0, 'train_loss': 1.035851001739502, 'epoch': 4.0})